# Training quadcopter dynamics

number of fully connected layers: 2
Depth of fully connected layers(number of output) = 512
use relu
Batch size 512
lr: 0.001
nEpoch: 20

neural network: input the current state and current input => outputs next state


In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib

from torch.utils.data import DataLoader
import torch.optim as optim

In [47]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Using PyTorch version: {}, Device: {}".format(torch.__version__, DEVICE))

Using PyTorch version: 1.12.1, Device: cpu


In [48]:
class nn_Dynamics(nn.Module):
    def __init__(self, input_state_dim, depth_fc_layer, output_state_dim):
        super(nn_Dynamics, self).__init__()
        self.fc1 = nn.Linear(input_state_dim, depth_fc_layer, bias=True).double()
        nn.init.xavier_normal_(self.fc1.weight)

        self.fc2 = nn.Linear(depth_fc_layer, depth_fc_layer, bias=True).double()
        nn.init.xavier_normal_(self.fc2.weight)
        
        self.output_layer = nn.Linear(depth_fc_layer, output_state_dim, bias=True).double()
        nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, input_states):
        x = self.fc1(input_states)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        output = self.output_layer(x)

        return output

In [49]:
state = np.genfromtxt('state_history.csv',delimiter=',')
input = np.genfromtxt('input_history.csv',delimiter=',')

current_state = state[:-1,:]
input = input[:-1,:]
next_state = state[1:,:]

dataX = np.append(current_state,input,axis=1)
dataY = next_state

train_data = []
for i in range(len(dataX)):
    train_data.append([dataX[i],dataY[i]])

trainloader = DataLoader(train_data, shuffle=True, batch_size=24)


In [50]:
for (X_train, y_train) in trainloader:
    print(X_train.size(),X_train.type())
    print(y_train.size(),y_train.type())
    break

torch.Size([24, 16]) torch.DoubleTensor
torch.Size([24, 12]) torch.DoubleTensor


In [51]:
model = nn_Dynamics(input_state_dim=16, depth_fc_layer=512, output_state_dim=12).to(DEVICE)
optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.MSELoss()

In [52]:
def train(nn_Dynamics, trainloader, optimizeer):
    model.train()
    training_loss_list = []
    train_loss = 0    

    for (state_inputs, next_states) in trainloader:
        state_inputs = state_inputs.to(DEVICE)
        next_states = next_states.to(DEVICE)
        optimizer.zero_grad()
        output = nn_Dynamics(state_inputs)
        loss = criterion(output, next_states)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        training_loss_list.append(train_loss)
    
    print(sum(training_loss_list))


In [54]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train(model, trainloader, optimizer)

10.331615970748686
12.96269165998383
10.311767611200358
14.321649384767237
9.825679227375241
3.618692672458381
3.2253581508414193
3.757046859490706
5.579842044038995
4.57942744163356
3.99690849117002
6.496032056214091
11.790708138144488
7.366344692703697
6.505735504779219
4.0908439573451405
4.627331346182234
9.805041963173881
6.09699621456753
9.330180681293086
11.83007341161839
5.164875249485742
5.7257527550620155
6.631530273818324
10.576626158752896
18.462124153022785
10.105590745865172
7.108636339723925
3.032956325306692
1.8157112111458755
1.818215654753011
2.09227382898952
6.515605619602836
33.79143961499399
4.746084541613979
4.798428356392463
7.12763777327447
4.079965418150535
2.2609685397757744
3.5250582325968582
7.45576582338109
10.263649877225564
9.60180235278855
4.785030572332917
3.7106216373607763
3.7549912522597917
3.936738147020903
8.707796925104255
7.528240074061032
7.568262474961299
3.4018653521620967
3.0900417209894098
3.606005827706184
5.375392822962275
7.926439853831686